# OCR for printed Alphanumeric characters

In [1]:
import numpy as np
import cv2
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import random
import keras.backend as K
K.set_image_data_format('channels_last')
import keras

Using TensorFlow backend.


In [2]:
X_train=[]
X_test=[]
Y_train=[]
Y_test=[]

# loop through the directories in font folder starting from Sample001 upto Sample062
for i in range(1,63):
    s="%03d"%i
    directory="/home/wolf/Documents/datasets/fonts/EnglishFnt/English/Fnt/Sample"+s
    # loop through the files in selected font folder starting from img001-00001.png upto img001-01016.png
    for j in range(1,1017):
        s2="%05d"%j
        img_name=directory+"/img"+s+"-"+s2+".png"
        img=cv2.imread(img_name)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_CUBIC)
        if j<=762:
            X_train.append(img)
        else:
            X_test.append(img)

In [3]:
X_train=np.array(X_train)
X_test=np.array(X_test)

# create the label dataset 
for i in range(0,62):
    for k in range(0,1016):
        temp=np.zeros(62)
        temp[i]=1
        if k<762:
            Y_train.append(temp)
        else:
            Y_test.append(temp)
            
            
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [4]:
img_rows, img_cols = 32, 32

In [5]:
input_shape = (img_rows, img_cols, 1)

In [6]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)

In [7]:
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [9]:
num_classes = 62

In [10]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 47244
number of test examples = 15748
X_train shape: (47244, 32, 32, 1)
Y_train shape: (47244, 62)
X_test shape: (15748, 32, 32, 1)
Y_test shape: (15748, 62)


In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [12]:
epochs = 20
batch_size = 128

In [14]:
from tensorflow.python.framework import tensor_util
def is_tensor(x):                                                                                                                                                      
    return tensor_util.is_tensor(x)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [ ]:
model.save('keras_alphanumeric_weights.h5')

In [ ]:
import matplotlib.pyplot as plt
from skimage.util import img_as_float
print(X_train[0].reshape(32,32).astype(int))
plt.imshow(X_train[0].reshape(32,32))

In [ ]:
from tensorflow import keras
reconstructed_model = keras.models.load_model('keras_alphanumeric_weights.h5')

In [ ]:
all_characters = list('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz')
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
predicted = reconstructed_model.predict(X_train[1300].reshape(1,32,32,1))
print(all_characters[np.argmax(predicted)])